---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_csv('university_towns.txt',sep=';',names=['RegionName'])
    df.insert(0, 'State', df['RegionName'].str.extract('(.*)\[edit\]', expand=False).ffill())
    df['RegionName'] = df['RegionName'].str.replace(r' \(.+$', '')
    df = df[~df['RegionName'].str.contains('\[edit\]')].reset_index(drop=True)
    
    return df

get_list_of_university_towns().head()

State    RegionName
0  Alabama        Auburn
1  Alabama      Florence
2  Alabama  Jacksonville
3  Alabama    Livingston
4  Alabama    Montevallo

In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls',
                       skiprows=5,
                       names=['Quarter','GDP in billions of chained 2009 dollars'],
                       parse_cols="E,G")
    
    df = df[df['Quarter']>='2000q1']
    
    df['GDP change'] = (df['GDP in billions of chained 2009 dollars']
                        .diff()
                        /df['GDP in billions of chained 2009 dollars']
                        .shift())*100

    df['GDP change'] = df['GDP change'].apply(lambda x: -1 if x<0 else 0)
    df['GDP change'] = df['GDP change']+df['GDP change'].shift()
    
    ans = df.loc[df['GDP change']==-2].iloc[0].Quarter
    
    return ans

get_recession_start()

'2008q4'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls',
                       skiprows=5,
                       names=['Quarter','GDP in billions of chained 2009 dollars'],
                       parse_cols="E,G")
    
    df = df[df['Quarter']>=get_recession_start()]

    df['GDP change'] = (df['GDP in billions of chained 2009 dollars']
                        .diff()
                        /df['GDP in billions of chained 2009 dollars']
                        .shift())*100
    
    df['GDP change'] = df['GDP change'].apply(lambda x: 1 if x>0 else 0)
    df['GDP change'] = df['GDP change']+df['GDP change'].shift()
    
    ans = df.loc[df['GDP change']==2].iloc[0].Quarter

    return ans

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    df = pd.read_excel('gdplev.xls',
                       skiprows=5,
                       names=['Quarter','GDP in billions of chained 2009 dollars'],
                       parse_cols="E,G")
    
#     df.set_index('Quarter',inplace=True)

    df = df.loc[(df['Quarter']>=get_recession_start()) & (df['Quarter']<=get_recession_end())]

    ans = df['Quarter'].loc[df['GDP in billions of chained 2009 dollars'] == df['GDP in billions of chained 2009 dollars'].min()]
    
    return ans.values[0]

get_recession_bottom()

'2009q2'

In [9]:
df = pd.read_csv('City_Zhvi_AllHomes.csv')

In [12]:
df = df.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)

In [16]:
df = df.drop(df.loc[:,'1996-04':'1999-12'],axis=1)

In [18]:
df = df.set_index(['State','RegionName'])

In [ ]:
for y in range (2000,2016):
    for q in range (1,4):
        df[(y,"q")]

In [19]:
df

2000-01    2000-02    2000-03    2000-04  \
State RegionName                                                        
NY    New York                   NaN        NaN        NaN        NaN   
CA    Los Angeles           204400.0   207000.0   209800.0   212300.0   
IL    Chicago               136800.0   138300.0   140100.0   141900.0   
PA    Philadelphia           52700.0    53100.0    53200.0    53400.0   
AZ    Phoenix               111000.0   111700.0   112800.0   113700.0   
NV    Las Vegas             131700.0   132600.0   133500.0   134100.0   
CA    San Diego             219200.0   222900.0   226600.0   230200.0   
TX    Dallas                 85100.0    84500.0    83800.0    83600.0   
CA    San Jose              364100.0   374000.0   384700.0   395700.0   
FL    Jacksonville           88000.0    88800.0    89000.0    88900.0   
CA    San Francisco         418700.0   430600.0   442200.0   453600.0   
TX    Austin                142500.0   143000.0   143400.0   143900.0   
MI    Detroit                64400.0    66200.0    67900.0    68900.0   
OH    Columbus               93800.0    94400.0    94900.0    95400.0   
TN    Memphis                72400.0    72500.0    72600.0    72900.0   
NC    Charlotte             126600.0   126900.0   127300.0   127800.0   
TX    El Paso                75900.0    76300.0    76600.0    76900.0   
MA    Boston                203100.0   206900.0   210800.0   214700.0   
WA    Seattle               246100.0   248800.0   250900.0   252900.0   
MD    Baltimore              59300.0    59700.0    60000.0    59800.0   
CO    Denver                160600.0   162200.0   163900.0   165800.0   
DC    Washington            136200.0   137900.0   139200.0   140900.0   
TN    Nashville             113100.0   113900.0   114500.0   115000.0   
WI    Milwaukee              77600.0    78100.0    78400.0    78600.0   
AZ    Tucson                101500.0   101900.0   102100.0   102500.0   
OR    Portland              152300.0   152800.0   153300.0   153800.0   
OK    Oklahoma City          76000.0    76400.0    76900.0    77200.0   
NE    Omaha                 112400.0   112800.0   113200.0   113400.0   
NM    Albuquerque           125300.0   125800.0   126500.0   126700.0   
CA    Fresno                 93300.0    94300.0    94700.0    94800.0   
...                              ...        ...        ...        ...   
TX    Granite Shoals             NaN        NaN        NaN        NaN   
MD    Piney Point           155300.0   156200.0   155500.0   154300.0   
WI    Maribel                    NaN        NaN        NaN        NaN   
ID    Middleton             106700.0   105600.0   105900.0   105500.0   
CO    Bennett               131600.0   133100.0   134000.0   134700.0   
NH    East Hampstead        160000.0   161800.0   163700.0   166300.0   
MO    Garden City                NaN        NaN        NaN        NaN   
AR    Mountainburg           54800.0    56800.0    59900.0    62500.0   
WI    Oostburg              106800.0   107300.0   107700.0   107400.0   
CA    Twin Peaks             96800.0    97000.0    98300.0    99800.0   
NY    Upper Brookville     1222500.0  1240300.0  1230100.0  1221900.0   
HI    Volcano                98500.0    97400.0   100200.0   103700.0   
SC    Wedgefield                 NaN        NaN        NaN        NaN   
MI    Williamston           157400.0   159400.0   160700.0   160900.0   
AR    Decatur                63600.0    63500.0    63700.0    64200.0   
TN    Briceville             39000.0    40200.0    40800.0    41100.0   
IN    Edgewood               91300.0    91800.0    92000.0    91500.0   
TN    Palmyra                    NaN        NaN        NaN        NaN   
MD    Saint Inigoes         148200.0   148400.0   147600.0   146700.0   
IN    Marysville                 NaN        NaN        NaN        NaN   
CA    Forest Falls          114200.0   113300.0   113100.0   114000.0   
MO    Bois D Arc            106600.0   108400.0   108400.0   107500.0   
VA    Henrico 

In [ ]:
,'1996-04':'1999-12'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return "ANSWER"

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"